In [1]:
def extractURL(sentence):
    subStr = sentence
    urlList = []

    if "@handle" in subStr:
        subStr = subStr.replace("@handle", "")
    
    http_targets = ["http://", "https://"]
    for target in http_targets:
        while (target in subStr):
            index = subStr.index(target)
            subStr = subStr[index:]
            tempURL = subStr.split()[0]
            subStr = subStr[len(tempURL):]
            tempURL = tempURL[len(target):]
            if len(tempURL) <= 5:
                continue
            if "http://" in tempURL:
                newURLList = tempURL.split("http://")
                if '' in newURLList:
                    newURLList.remove('')
                for newURLListItem in newURLList:
                    if '/' in newURLListItem:
                        newURLListItem = newURLListItem.split('/')[0]
                        part = newURLListItem.split('.')
                        urlList.append("{0}".format(part[-2]))
                continue
            elif "http:/" in tempURL:
                tempURL = tempURL.replace("http:/", '')
                if len(tempURL) < 5:
                    continue
            if '/' in tempURL:
                tempURL = tempURL.split('/')[0]
            part = tempURL.split('.')
            while '' in part:
                part.remove('')
            if len(part) < 2:
                continue
            tempURL = "{0}".format(part[-2])
            if '?' in tempURL:
                part = tempURL.split("?")
                while '' in part:
                    part.remove('')
                tempURL = part[0]
            urlList.append(tempURL)
    return urlList

In [2]:
def extractEmail(sentence):
    email_suffix = ["@handle.com", "@handle.net"]
    for surfix in email_suffix:
        if surfix in sentence:
            email = sentence.split(surfix)[0]    
            if "" != email and ("@" not in email):
                return [email]
    return []

In [3]:
remove_words = ["@handle", "http", "RT"]
urlRemoveList = ["bit.ly", "ow.ly", "tinyurl", "short.to", "s1z.us", "s3z.us", ]


def pre_process(sentence, setence_len_limit=5):
    sentence = sentence.split()
    remain_tokens = []
    target_remove = set()
    for token in sentence:
        lowerSubStr = token.lower()
        urlRemoveFlag = False
        for urlRemove in urlRemoveList:
            if urlRemove in lowerSubStr:
                urlRemoveFlag = True
                break
        if urlRemoveFlag:
            continue
        if ("RT" in token) and (len(token) < 4):
            continue
        if ("http" in token.lower()):
            urlList = extractURL(token)
            remain_tokens += urlList
            continue
        if ("@handle" in token):
            email_list = extractEmail(token)
            remain_tokens += email_list
            continue
        remain_tokens.append(token)
            
    sentence = remain_tokens
    while "" in sentence:
        sentence.remove("")
    while "@" in sentence:
        sentence.remove("@")
    sentence_length = len(sentence)
    if sentence_length < setence_len_limit:
        return ("", 0)
    else:
        sentence = ' '.join(sentence)
        return (sentence, sentence_length)


In [4]:
import csv
import collections
import numpy as np
import pandas as pd
import random
import pickle
random.seed(10)

train_file_path = "../data/train_tweets.txt"
train_dict = collections.defaultdict(list)

length_array = []
id_counter = set()
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip().split("\t")
        id = int(row[0])
        id_counter.add(id)
        (instance, sentence_length) = pre_process(row[1])
        if not instance == "":
            train_dict[id].append(instance)
            length_array.append(sentence_length)
    print("Total rows: %d" % i)
    
print("Total ids: %d" % len(id_counter))
print("Total ids after filtering: %d" % len(train_dict))


Total rows: 328931
Total ids: 9297
Total ids after filtering: 9287


In [5]:
def splitTrainDev(trainDict, tweets_num_limit=10):
    dev_split = 0.1
    train_split = 1 - dev_split

    dev_set_dict = {}
    train_set_dict = {}

    for id in trainDict:
        target_list = trainDict[id]
        length = len(target_list)
        if length < tweets_num_limit:
            continue
        random.shuffle(target_list)
        split = int(np.ceil(length*dev_split))
        dev_set_dict[id] = target_list[:split]
        train_set_dict[id] = target_list[split:length]
    #     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

    print(len(dev_set_dict), len(train_set_dict), len(trainDict))
    return (dev_set_dict, train_set_dict)

In [6]:
def save_to_file(target_dict, file_path):
    id_list = []
    sentence_list = []
    for id in target_dict:
        for sentence in target_dict[id]:
            id_list.append(id)
            sentence_list.append(sentence)
      
    id_list = np.array(id_list)
    sentence_list = np.array(sentence_list)
    random_index = np.array(range(len(sentence_list)))
    random.shuffle(random_index)
    id_list = id_list[random_index]
    sentence_list = sentence_list[random_index]
    
    dataframe = pd.DataFrame({'id':id_list,'sentence':sentence_list})
    dataframe.to_csv(file_path,index=False,sep='\t',header=None)
    print(len(id_list))
    return

def save_idx(target_dict, file_path):
    # Build IDX and Save
    idx = {}
    for i, id in enumerate(target_dict):
        idx[id] = i
    print(len(idx))
    with open(file_path, 'wb') as handle:
        pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
version = "v2_1"
dev_set_path = '../data/{0}/dev_set_{0}.txt'.format(version)
train_set_path = '../data/{0}/train_set_{0}.txt'.format(version)
idx_file_path = '../data/{0}/{0}_idx.pickle'.format(version)

dev_set_dict, train_set_dict = splitTrainDev(train_dict)

save_to_file(dev_set_dict, dev_set_path)
with open(train_set_path, 'wb') as handle:
    pickle.dump(train_set_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
save_idx(train_set_dict, idx_file_path)

8791 8791 9287
32827
8791
